In [31]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input#Added dash_table to create the table
#Imported dcc because it includes the dcc.Graph to render the interactive graph
#Included callback, output, and input to edit the graph
import pandas as pd #Import pandas to read the csv file
import plotly.express as px #Import plotly to create the graph

# Incorporate data
df = pd.read_csv(r'C:\Users\zjriv\Python_Programming\EGR_598_Python\Project\Kaggle - Supply Chain\supply_chain_data.csv')
#Notice pd.read is the same as what you're used to

# Define available options for x and y axis for each chart type
chart_options = {
    'bar': { #Options for bar chart
        'x': ['Product type', 'Location', 'SKU',  'Shipping carriers'], #SKU is each individual product we sell
        'y': ['Revenue generated', 'Manufacturing costs', 'Stock levels', 'Shipping costs']
    },
    'pie': { #Options for pie chart
        'x': ['Product type', 'Location', 'Customer demographics'],
        'y': ['Number of products sold', 'Costs']
    },
    'box': { #Options for box plot
        'x': ['Product type'],
        'y': ['Defect rates']
    }
}

# Initialize the app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.Div(children='Supply Chain Data'), # Title of the whole program
    html.Hr(),
    html.Label('Select X-axis Data:'), # Title for x-axis selection
    dcc.RadioItems(
        id='x-axis-selector',
        value='Product type', #By default, it will be on what value equals ('Product type' in this case)
        labelStyle={'display': 'block'}  #The id name is used later for the callback Input
    ),
    html.Br(),
    html.Label('Select Y-axis Data:'), # Title for y-axis selection
    dcc.RadioItems(
        id='y-axis-selector',
        value='Revenue generated',
        labelStyle={'display': 'block'}  
    ),
    html.Br(),
    html.Label('Select Chart Type:'),
    dcc.Dropdown(
        id='chart-type-selector',
        options=[
            {'label': 'Bar Chart', 'value': 'bar'},
            {'label': 'Pie Chart', 'value': 'pie'},
            {'label': 'Box Plot', 'value': 'box'}  # Adding the box plot option
        ],
        value='bar', 
        clearable=False
    ),
    html.Div([
        dcc.Graph(figure={}, id='selected-chart')  #Can comment out this line and copy it later as a way to seperate
    ], style={'display': 'inline-block', 'width': '100%'}),
    dash_table.DataTable(data=df.to_dict('records'), page_size=6)
])

# Update options for x and y axes based on selected chart type
@app.callback(
    [Output('x-axis-selector', 'options'),
     Output('y-axis-selector', 'options')],
    [Input('chart-type-selector', 'value')]
)
def update_axis_options(chart_type):
    x_options = [{'label': option, 'value': option} for option in chart_options[chart_type]['x']]
    y_options = [{'label': option, 'value': option} for option in chart_options[chart_type]['y']]
    return x_options, y_options

# Update the selected chart based on user input
@app.callback(
    Output(component_id='selected-chart', component_property='figure'),
    [Input(component_id='x-axis-selector', component_property='value'),
     Input(component_id='y-axis-selector', component_property='value'),
     Input(component_id='chart-type-selector', component_property='value')]
)
def update_selected_chart(x_data, y_data, chart_type):
    #Seperate each chart type through if statements
    if chart_type == 'bar':
        fig = px.bar(df, x=x_data, y=y_data, barmode='group', title='Bar Chart')
    elif chart_type == 'pie':
        fig = px.pie(df, values=y_data, names=x_data, title='Pie Chart')
    elif chart_type == 'box':
        fig = px.box(df, x=x_data, y=y_data, title='Box Plot')
    else:
        fig = px.bar(df, x=x_data, y=y_data, barmode='group', title='Bar Chart')  # Default to bar chart
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [26]:
Locations = df.set_index(['Location'])
Locations[['Number of products sold', 'Manufacturing costs', 'Revenue generated']].groupby(['Location']).sum()

,Number of products sold,Manufacturing costs,Revenue generated
Location,,,
Bangalore,5420,1115.91,102601.73
Chennai,8768,1022.88,119142.84
Delhi,9715,733.19,81027.72
Kolkata,12770,1046.56,137077.55
Mumbai,9426,808.10,137755.02
